In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pandas_datareader.data as web
import time
import tensorflow
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import logging
import math
import os
from sklearn.preprocessing import MinMaxScaler, scale
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras.models import load_model
from tqdm import tqdm_notebook

Using TensorFlow backend.


In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

# 데이터 불러오기 / 데이터 정리

In [3]:
IT_finance = pd.read_excel('./data/dongboo/IT_TOTAL_재무사항.xlsx')
Medicine_finance = pd.read_excel('./data/dongboo/제약_TOTAL_재무사항.xlsx')

In [4]:
IT_stock = pd.read_excel('./data/dongboo/IT_TOTAL_주가.xlsx')
Medicine_stock = pd.read_excel('./data/dongboo/제약_TOTAL_주가.xlsx')

In [5]:
IT = pd.merge(IT_finance,IT_stock,how = 'left', on = ['회사명','거래소코드','회계년도'])
Medicine = pd.merge(Medicine_finance,Medicine_stock,how = 'left', on = ['회사명','거래소코드','회계년도'])

# 회사별로 분리

In [6]:
lg = IT[IT['회사명'] == 'LG이노텍(주)']
sdi = IT[IT['회사명'] == '삼성SDI(주)']
ssem = IT[IT['회사명'] == '삼성전기(주)']
hynix = IT[IT['회사명'] == '에스케이하이닉스(주)']

In [7]:
greencross = Medicine[Medicine['회사명'] == '(주)녹십자']
daewoong = Medicine[Medicine['회사명'] == '(주)대웅제약']
yoohan = Medicine[Medicine['회사명'] == '(주)유한양행']
hanmi = Medicine[Medicine['회사명'] == '한미약품(주)']

In [8]:
def classify(stock):
    
    # 회계년도 순으로 정렬 및 인덱스 초기화
    stock.sort_values('회계년도', ascending= True, inplace = True)
    
    stock.reset_index(drop = True, inplace = True)
    
    
    # 새로운 변수 추가
    
    stock['3개월후종가'] = stock['종가'][1:].reset_index(drop=True)
    
    stock['분기수익률'] = (stock['3개월후종가'] - stock['종가']) / stock['종가']
    
    stock['수익률상승하락'] = stock['분기수익률']
    
    for i in range(len(stock)):
        if stock['분기수익률'][i] < 0.005:
            stock['수익률상승하락'][i] = 0
        else:
            stock['수익률상승하락'][i] = 1
        
       
    return stock

In [9]:
classify(lg)
classify(sdi)
classify(ssem)
classify(hynix)

classify(greencross)
classify(daewoong)
classify(yoohan)
classify(hanmi)

,회사명,거래소코드,회계년도,자산(*),자본(*),* 수권주식수,*주당 액면가액,* 발행한 주식총수(*),보통주,우선주,지배기업주주지분(*),자본금(*),보통주자본금,우선주자본금,신종자본증권,자본잉여금(*),주식발행초과금,배당건설이자,주식매입선택권,감자차익,보험차익,합병차익,자산수증이익,채무면제이익,자기주식처분이익,전환권대가,신주인수권대가,교환권대가,전환권재매입이익,기타자본잉여금,재평가적립금(2001년 이전),기타자본(*),자기주식,자기주식처분손실,주식할인발행차금,주식매입선택권.1,감자차손,파생상품평가이익,파생상품평가손실,전환권재매입손실,기타,기타포괄손익누계액(*),금융자산평가손익,매도가능금융자산평가손익,관계기업 등 기타포괄손익,환산 관련 외환차이 적립금(환율변동차이),해외사업장순투자위험회피 적립금,확정급여제도의 보험수리적 손익 적립금,확정급여제도의 재측정요소,현금흐름위험회피 적립금,주식기준보상 적립금,재평가잉여금,기타.1,이익잉여금(결손금)(*),* (지배)당기순손익,이익준비금,재무구조개선적립금,기업합리화적립금,기타법정적립금,재평가손익,보험수리적손익,기타이익잉여금,미처분이익잉여금(결손금),*전기이월미처분이익잉여금(결손금),비지배주주지분,* (비지배)당기순손익,*비지배주주지분초과손실액,부채(*),* (정상)영업손익(보고서기재)(3개월),* (정상)영업손익(계산수치)(3개월),기타이익(*)(3개월),이자수익(3개월),배당금수익(3개월),외환거래이익(*)(3개월),외환차익(3개월),외환환산이익(3개월),기타외화거래이익(3개월),보험차익(3개월),퇴직연금운용이익(3개월),전기오류수정이익(3개월),기타(3개월),기타손실(*)(3개월),이자비용(3개월),외환거래손실(*)(3개월),외환차손(3개월),외화환산손실(3개월),기타외환거래손실(3개월),법인세추납액(3개월),퇴직연금운용손실(3개월),전기오류수정손실(3개월),기타(3개월).1,법인세비용차감전순이익(손실)(3개월),법인세비용(3개월),종속회사 매수일전 순손익(3개월),처분된 종속회사 순손익(3개월),계속영업이익(손실)(3개월),중단영업이익(손실)(3개월),*중단영업손익 법인세효과(3개월),당기순이익(손실)(3개월),기타포괄손익(*)(3개월),금융자산평가손익(3개월),매도가능금융자산평가손익(3개월),관계기업 등 기타포괄손익(3개월),환산 관련 외환차이(환율변동차이)(3개월),해외사업장순투자위험회피(3개월),현금흐름위험회피 적립금(3개월),확정급여제도의 보험수리적손익(3개월),확정급여제도의 재측정요소(3개월),주식기준보상 적립금(3개월),재평가손익(3개월),기타(3개월).2,기타포괄손익관련 법인세(3개월),총포괄손익(3개월),*(당기순손익 귀속)지배기업주주지분(3개월),*계속영업손익(3개월),*중단영업손익(3개월),*(당기순손익 귀속)비지배주주지분(3개월),*계속영업손익(3개월).1,*중단영업손익(3개월).1,*(총포괄손익 귀속)지배기업주주지분(3개월),*(총포괄손익 귀속)비지배주주지분(3개월),*주당계속영업이익(3개월),*주당순손익(3개월),*희석주당계속영업이익(3개월),*희석주당순손익(3개월),*(지배기업주주지분)주당계속영업이익(3개월),*(지배기업주주지분)주당순손익(3개월),*(지배기업주주지분)희석주당계속영업이익(3개월),*(지배기업주주지분)희석주당순손익(3개월),* 성격별 비용계정(*)(3개월),제품과 재공품의 변동(3개월),기업이 수행한 용역으로서 자본화되어 있는 부분(3개월),원재료와 저장품(소모품)의 사용액(3개월),상품의 판매(3개월),기타원가(3개월),종업원 급여비용(3개월),감가상각비와 기타상각비 및 손상차손(3개월),세금과공과(3개월),대손상각비(3개월),물류비(운송보관)(3개월),광고 및 판매촉진비(3개월),임차료 및 리스료(3개월),경상연구개발비(3개월),기타비용(3개월),기말자본금(*),기초자본금,유상증자(감자),무상증자(감자),주식매수선택권,전환사채,신주인수권부사채,주식배당,합병분할(영업양수도 등)로 인한 변동,회계정책변경누적효과,오류수정 등,연결범위의 변동,외화환산차이,기타.2,기말신종자본증권(*),기초신종자본증권,신종자본증권의 발행,기타.3,기말자본잉여금(*),기초자본잉여금,유상증자(감자).1,무상증자(감자).1,주식발행초과금.1,주식매수선택권.1,감자차손익,보험차익.1,자기주식처분손익,전환권대가(전환사채),신주인수권대가(신주인수권부사채),교환권대가(교환사채),전환권재매입손익,결손금처리,지분법자본잉여금,합병분할(영업양수도 등)로 인한 변동.1,회계정책변경누적효과.1,오류수정 등.1,연결범위의 변동.1,외화환산차이.1,기타.4,기말기타자본(*),기초기타자본,자기주식.1,자기주식처분손실.1,주식할인발행차금.1,주식매입선택권.2,파생상품평가이익.1,파생상품평가손실.1,출자전환채무,신주청약증거금,감자차손익.1,전환권재매입손익.1,지분법자본조정,합병분할(영업양수도 등)로 인한 변동.2,회계정책변경누적효과.2,오류수정 등.2,연결범위의 변동.2,외화환산차이.2,기타.5,기말기타포괄손익누계액(*),기초기타포괄손익누계액,금융자산평가손익.1,매도가능금융자산 평가손익,관계기업 등 기타포괄손익.1,환산 관련 외환차이 적립금(환율변동차이).1,해외사업장순투자위험회피 적립금.1,확정급여제도의 보험수리적 손익 적립금.1,확정급여제도의 재측정요소.1,현금흐름위험회피 적립금.1,주식기준보상 적립금.1,재평가잉여금.1,합병분할(영업양수도 등)로 인한 변동.3,회계정책변경누적효과.3,오류수정 등.3,연결범위의 변동.3,외화환산차이.3,기타.6,기말이익잉여금(결손금)(*),처분후 이익잉여금(결손금)(*),수정후 이익잉여금(결손금)(*),기초이익잉여금(결손금),회계정책변경누적효과.4,전기오류수정손익,기타.7,(연차배당),(신종자본증권 분배금),주식할인발행차금의 상각,자기주식처분손실의 상각,결손금처리.1,기타이익잉여금처분액,(중간배당),확정급여제도의 재측정요소.2,기타포괄손익,당기순이익(총포괄손익),보험수리적손익.1,재평가손익.1,지분법이익잉여금,합병분할(영업양수도 등)로 인한 변동.4,연결범위의 변동.4,외화환산차이.4,기타.8,기말비지배주주지분(*),처분후 비지배주주지분(*),수정후 비지배주주지분(*),기초 비지배주주지분,회계정책변경누적효과.5,전기오류수정손익.1,기타.9,(종속회사의 배당금),기타.10,(중간배당).1,당기순이익(총포괄손익).1,재평가잉여금.2,금융자산평가손익.2,매도가능금융자산 평가손익.1,관계기업 등 지분법평가손익,환산 관련 외환차이 적립금(환율변동차이).2,해외사업장순투자위험회피 적립금.2,확정급여제도의 보험수리적 손익 적립금.2,확정급여제도의 재측정요소.3,현금흐름위험회피 적립금.2,주식기준보상 적립금.2,기타포괄손익누계액,합병분할로 인한 변동,연결범위의 변동.5,외화환산차이.5,기타.11,합계,영업활동으로 인한 현금흐름(간접법)(*),*(직접법)총현금유입(*),재화의 판매와 용역제공에 따른 현금유입,"수입보험료, 보험금, 연금 및 기타 급부금관련 현금유입",이자수취액,배당금수취액,법인세환급,기타현금유입액,*(직접법)총현금유출(*),종업원과 관련하여 직·간접으로 발생하는 현금유출,"수입보험료, 보험금, 연금 및 기타 급부금관련 현금유출",이자비용유출액,배당금지급액,법인세납부,단기매매목적으로 보유하는 계약에서 발생하는 현금유출,기타현금유출액,당기순손익,법인세비용차감전순손익,현금 유출이 없는 비용 등 가산(*),전기오류수정손익.2,비지배주주지분순손익,보험수리적손실,기타.12,(현금 유입이 없는 수익 등 차감)(*),전기오류수정손익.3,비지배주주지분순손익.1,보험수리적이익,기타.13,영업활동으로 인한 자산부채 변동(운전자본 변동)(*),자산의 감소(증가)(*),부채의 증가(감소)(*),정부보조금 등의 변동,기타 운전자본의 변동,*영업에서 창출된 현금흐름,이자수입,이자지급(-),배당금수입,배당금지급(-),법인세환입,법인세납부(-),중단영업관련 현금흐름,투자활동으로 인한 현금흐름(*),투자활동으로 인한 현금유입액(*),정부보조금 등의 변동.1,종속기업 및 기타사업의 지배력 관련한 현금흐름,합병분할(양수도등)으로 인한 현금유입액,기타투자활동으로 인한 현금유입액,(투자활동으로 인한 현금유출액)(*),정부보조금 등의 변동.2,종속기업 및 기타사업의 지배력 관련한 현금흐름.1,

In [10]:
lg = lg.iloc[:-4,:]
sdi = sdi.iloc[:-4,:]
ssem = ssem.iloc[:-4,:]
hynix = hynix.iloc[:-4,:]

greencross = greencross.iloc[:-4,:]
daewoong = daewoong.iloc[:-4,:]
yoohan = yoohan.iloc[:-4,:]
hanmi = hanmi.iloc[:-4,:]

# 앞서 유한양행만 상관관계 분석: 변수 <3개월 후 종가>와의 상관관계 분석

In [11]:
df_yoohan = pd.DataFrame(yoohan.corr()['3개월후종가'].reset_index().sort_values('3개월후종가', ascending = False))
df_yoohan.columns = ['지표','3개월후종가']
df_yoohan

,지표,3개월후종가
379,3개월후종가,1.000000
376,종가,0.706961
373,최고가,0.677100
375,산술평균,0.670699
374,최저가,0.661262
372,거래일수,0.647462
65,부채(*),0.639543
225,(연차배당),0.620389
124,*(지배기업주주지분)주당순손익(3개월),0.609356
123,*(지배기업주주지분)주당계속영업이익(3개월),0.609356


# 산업별 변수 <3개월 후 종가>와의 상관관계 분석

In [12]:
# 밑에 함수로 해보기!!!!

In [13]:
lg_corr = lg.corr(method = 'pearson')
df_lg = pd.DataFrame(lg_corr['3개월후종가']).reset_index().sort_values('3개월후종가', ascending = False)
df_lg.columns = ['지표','3개월후종가']

sdi_corr = sdi.corr(method = 'pearson')
df_sdi = pd.DataFrame(sdi_corr['3개월후종가']).reset_index().sort_values('3개월후종가', ascending = False)
df_sdi.columns = ['지표','3개월후종가']

ssem_corr = ssem.corr(method = 'pearson')
df_ssem = pd.DataFrame(ssem_corr['3개월후종가']).reset_index().sort_values('3개월후종가', ascending = False)
df_ssem.columns = ['지표','3개월후종가']

hynix_corr = hynix.corr(method = 'pearson')
df_hynix = pd.DataFrame(hynix_corr['3개월후종가']).reset_index().sort_values('3개월후종가', ascending = False)
df_hynix.columns = ['지표','3개월후종가']

# 산업별 상관관계 데이터프레임

In [14]:
pd.concat([df_lg.reset_index(drop = True), df_sdi.reset_index(drop = True), df_ssem.reset_index(drop = True), df_hynix.reset_index(drop = True)],axis = 1)

,지표,3개월후종가,지표,3개월후종가,지표,3개월후종가,지표,3개월후종가
0,3개월후종가,1.000000,3개월후종가,1.000000,3개월후종가,1.000000,3개월후종가,1.000000
1,종가,0.750333,종가,0.712030,종가,0.825649,종가,0.915672
2,최고가,0.699947,최저가,0.662528,최고가,0.805077,* (정상)영업손익(계산수치),0.894498
3,산술평균,0.676859,산술평균,0.654951,산술평균,0.794202,최저가,0.892365
4,최저가,0.649876,최고가,0.652450,최저가,0.784358,산술평균,0.891709
5,거래대금,0.629034,정부보조금 등의 변동.1,0.561688,기초자본잉여금,0.754838,* (정상)영업손익(보고서기재),0.890801
6,확정급여제도의 재측정요소.3,0.591835,매도가능금융자산 평가손익.1,0.533294,주식매입선택권.1,0.735921,최고가,0.889801
7,법인세비용차감전순이익(손실),0.584854,이자지급(-),0.447417,거래대금,0.730648,법인세비용차감전순이익(손실),0.882430
8,*(당기순손익 귀속)지배기업주주지분,0.574488,분기수익률,0.443430,자본잉여금(*),0.708995,*(지배기업주주지분)주당계속영업이익,0.868868
9,당기순이익(손실),0.574432,주식발행초과금.1,0.388546,기말자본잉여금(*),0.708995,*(지배기업주주지분)주당순손익,0.868868


In [15]:
list_corr = []
order = 15 # 기업별로 상위 15개 상관관계 변수 추출

for i in range(len(df_lg.index[:order])):
    list_corr.append(df_lg.index[:order][i])
    list_corr.append(df_sdi.index[:order][i])
    list_corr.append(df_ssem.index[:order][i])
    list_corr.append(df_hynix.index[:order][i])

variable = pd.Series(list_corr).value_counts()
variable = pd.DataFrame(variable).reset_index()
variable.columns = ['var_index', 'number']

In [16]:
var = []
for i in range(len(variable)):
    var.append(df_lg[df_lg.index == variable['var_index'][i]]['지표'].values[0])
var = pd.DataFrame(var)
var.columns = ['var']

In [17]:
corr = pd.concat([variable,var], axis = 1)
corr = corr[['var_index','var','number']]
corr.head(20)

,var_index,var,number
0,379,3개월후종가,4
1,376,종가,4
2,375,산술평균,4
3,374,최저가,4
4,373,최고가,4
5,93,계속영업이익(손실),3
6,66,* (정상)영업손익(보고서기재),2
7,123,*(지배기업주주지분)주당계속영업이익,2
8,67,* (정상)영업손익(계산수치),2
9,96,당기순이익(손실),2


# 위 자료를 보고 변수 선정

In [18]:
deep_lg = lg[corr['var'][5:15]]
deep_lg['수익률상승하락'] = lg['수익률상승하락']

In [19]:
deep_lg.iloc[:,:-1]

,계속영업이익(손실),* (정상)영업손익(보고서기재),*(지배기업주주지분)주당계속영업이익,* (정상)영업손익(계산수치),당기순이익(손실),거래대금,*(당기순손익 귀속)지배기업주주지분,주식매입선택권.1,법인세비용차감전순이익(손실),*(지배기업주주지분)주당순손익
0,41501253,42395332,2421,41554974,47401856,6.443598e+09,47423280,0,41080355,2765
1,37610969,86912500,2102,79851762,101442004,1.009753e+10,101219879,0,62848450,5689
2,66807768,63152758,3319,68547655,66638920,1.165923e+10,66598571,0,59874975,3311
3,-19953483,-36004758,-1222,-36793272,-19651440,5.498511e+09,-19650321,0,-55127153,-1286
4,-9530267,-7369064,-473,-8786085,-9530267,5.974884e+09,-9529000,0,-26674417,-473
5,-6975403,5914429,-346,9623556,-6975403,1.997593e+09,-6974465,0,-15194170,-346
6,-35552775,-5366696,-1764,-15828492,-35552775,2.081236e+09,-35551689,0,-43309363,-1764
7,-93316075,-60014817,-4628,-27898047,-93316075,1.931456e+09,-93302093,0,-89294631,-4628
8,-2432253,23863085,-121,18258822,-2432253,4.785877e+09,-2431835,0,43058,-121
9,4459276,19046931,221,15243876,4459276,1.832339e+09,4459358,0,-7965601,221


In [20]:
# 표준화

from sklearn.preprocessing import StandardScaler

scaler_lg = StandardScaler()
scale_lg = pd.DataFrame(scaler_lg.fit_transform(deep_lg.iloc[:,:-1]))


# 원래 값으로 변환
# scaler_lg.inverse_transform(scale_lg)

In [21]:
scale_lg

,0,1,2,3,4,5,6,7,8,9
0,0.924697,0.172468,1.225899,0.141516,0.936018,0.800951,0.937218,0.0,0.669471,1.187400
1,0.814749,1.287341,1.034305,1.148849,2.326967,2.122851,2.322583,0.0,1.176267,2.723248
2,1.639913,0.692310,1.765245,0.851513,1.431164,2.687834,1.431018,0.0,1.107040,1.474190
3,-0.812145,-1.790956,-0.962116,-1.919304,-0.789879,0.459043,-0.790056,0.0,-1.570397,-0.940411
4,-0.517562,-1.073814,-0.512261,-1.182622,-0.529368,0.631382,-0.529413,0.0,-0.907971,-0.513378
5,-0.445357,-0.741147,-0.435983,-0.698387,-0.463608,-0.807501,-0.463629,0.0,-0.640692,-0.446671
6,-1.253014,-1.023667,-1.287646,-1.367860,-1.199166,-0.777241,-1.199547,0.0,-1.295260,-1.191484
7,-2.885527,-2.392256,-3.007787,-1.685330,-2.685946,-0.831428,-2.686731,0.0,-2.365872,-2.695816
8,-0.316957,-0.291647,-0.300847,-0.471251,-0.346671,0.201230,-0.346647,0.0,-0.285945,-0.328488
9,-0.122188,-0.412261,-0.095439,-0.550554,-0.169289,-0.867286,-0.169186,0.0,-0.472399,-0.148851


# 샤비어함수 및 히든레이어 5개

In [22]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        # 6 was used in the paper.
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        # 3 gives us approximately the same limints as above since this repicks
        # values greater than 2 standard deviations from the mean.
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [76]:
# Review : Learning rate and Evaluation
import tensorflow as tf
import random
import matplotlib.pyplot as plt

tf.set_random_seed(777)  # reproducibility

tf.reset_default_graph()




x_data = scale_lg
y_data = deep_lg.iloc[:,[-1]]

placeholder_num = len(x_data.columns)

# parameters
learning_rate = 0.001


X = tf.placeholder(tf.float32, [None, placeholder_num])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)


W1 = tf.get_variable("W1", shape=[placeholder_num, 32], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([32]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)


W2 = tf.get_variable("W2", shape=[32, 64], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[64, 64], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([64]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)


W4 = tf.get_variable("W4", shape=[64, 64], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([64]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)


W5 = tf.get_variable("W5", shape=[64, 1], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([1]))
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)

# hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(L4, W5) + b5)

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                       tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [31]:
# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in tqdm_notebook(range(2001)):
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_data, Y: y_data, keep_prob: 1.0})       
        if step % 200 == 0 or step < 10 :
            print("Step : {} \t Cost : {}".format(step, cost_val))    

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                      feed_dict={X: x_data, Y: y_data, keep_prob: 1.0})

Step : 0 	 Cost : 0.8639687895774841
Step : 1 	 Cost : 0.7579033374786377
Step : 2 	 Cost : 0.7157838940620422
Step : 3 	 Cost : 0.6993953585624695
Step : 4 	 Cost : 0.6925259232521057
Step : 5 	 Cost : 0.689247190952301
Step : 6 	 Cost : 0.6874915957450867
Step : 7 	 Cost : 0.6863619089126587
Step : 8 	 Cost : 0.6855034232139587
Step : 9 	 Cost : 0.6847353577613831
Step : 2000 	 Cost : 0.23567773401737213



In [67]:
predict_stock = pd.DataFrame(c)
predict_stock.columns = ['예측치']

In [71]:
pd.concat([y_data, predict_stock],axis = 1)

,수익률상승하락,예측치
0,1.0,1.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,1.0,1.0
7,1.0,1.0
8,0.0,0.0
9,0.0,1.0


# Validation

In [85]:

# Review : Learning rate and Evaluation
import tensorflow as tf
import random
import matplotlib.pyplot as plt

tf.set_random_seed(777)  # reproducibility

tf.reset_default_graph()




x_data = scale_lg.iloc[:-4,:]
y_data = deep_lg.iloc[:-4,[-1]]

placeholder_num = len(x_data.columns)

# parameters
learning_rate = 0.001


X = tf.placeholder(tf.float32, [None, placeholder_num])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)


W1 = tf.get_variable("W1", shape=[placeholder_num, 32], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([32]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)


W2 = tf.get_variable("W2", shape=[32, 64], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[64, 64], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([64]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)


W4 = tf.get_variable("W4", shape=[64, 64], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([64]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)


W5 = tf.get_variable("W5", shape=[64, 1], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([1]))
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)

# hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(L4, W5) + b5)

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                       tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [92]:
# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in tqdm_notebook(range(2401)):
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_data.iloc[:-4,:], Y: y_data.iloc[:-4,:], keep_prob: 1.0})       
        if step % 200 == 0 or step < 10 :
            print("Step : {} \t Cost : {}".format(step, cost_val))    

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data.iloc[-4:,:], Y: y_data.iloc[-4:,:], keep_prob: 1.0})

Step : 0 	 Cost : 0.9917289018630981
Step : 1 	 Cost : 0.8852556347846985
Step : 2 	 Cost : 0.8162601590156555
Step : 3 	 Cost : 0.7729681134223938
Step : 4 	 Cost : 0.7467948198318481
Step : 5 	 Cost : 0.7308642268180847
Step : 6 	 Cost : 0.7209902405738831
Step : 7 	 Cost : 0.714552104473114
Step : 8 	 Cost : 0.710110068321228
Step : 9 	 Cost : 0.7066889405250549
Step : 200 	 Cost : 0.6129094958305359
Step : 400 	 Cost : 0.5681749582290649
Step : 600 	 Cost : 0.5281556844711304
Step : 800 	 Cost : 0.4911474883556366
Step : 1000 	 Cost : 0.450756311416626
Step : 1200 	 Cost : 0.4028601348400116
Step : 1400 	 Cost : 0.3413779139518738
Step : 1600 	 Cost : 0.27021849155426025
Step : 1800 	 Cost : 0.19843891263008118
Step : 2000 	 Cost : 0.13497619330883026
Step : 2200 	 Cost : 0.08916004747152328
Step : 2400 	 Cost : 0.05656389519572258



In [98]:
print('모델 정확도: ', a * 100 , '%')

모델 정확도:  25.0 %


In [99]:
y_data.iloc[-4:,:]

,수익률상승하락
23,0.0
24,0.0
25,0.0
26,1.0


In [101]:
c

array([[1.],
       [0.],
       [1.],
       [0.]], dtype=float32)